- Advanced Pandas args to catch errors in data files
- Datetime hell (excel vs txt files)
- Pandas quick plots and stats
- Sum columns (header hygiene, . vs [])
- Volume to mass flow
- Averaging data 
- Static plot (matplotlib) and dynamic plot (Plotly)


- Mass-ballance (data handling in Pandas, volume flow to mass flow, sum seps)
- Pandas and numpy in computer memory and .copy() pointers vs copies
- Clean steamfield data (when to excel and when to Python)
